In [24]:
import matplotlib.pyplot as plt
% matplotlib inline

In [25]:
import nlopt
import autograd.numpy as np
import autograd
import flux_center

In [26]:
def penalty(x):
    return sum(x*(1-x))

def grad_penalty(x):
    return (1 - 2 * x)

def thresholding(x, eta=0.5, beta = 1):
    return (np.tanh(eta*beta) + np.tanh((x - eta)*beta))/(np.tanh(eta*beta) + np.tanh((1 - eta)*beta))

grad_thresholding = autograd.elementwise_grad(thresholding)

In [27]:
def optimization_withthresholdingNpenalty(x1, coefpenalty=0., constraintpores = 14.5, eta=0.2, beta = 1, debug = False):

    while not np.all(np.isclose( 1.*(thresholding(x1, eta=eta, beta=beta)>eta), thresholding(x1, eta=eta, beta=eta),atol=1e-01)):
        if debug:
            print("beta=", beta)
        def myfunc(x, grad):
            flux, gradval = flux_center.two_points_flux_normalized_grad(np.reshape(thresholding(x, eta=eta, beta=beta),(1, 25)))
            gradval = gradval*grad_thresholding(x, eta=eta, beta=beta)  - grad_thresholding(x, eta=eta, beta=beta)*grad_penalty(thresholding(x, eta=eta, beta=beta))*coefpenalty
            if grad.size > 0:
                grad[:] = np.ravel(gradval[:]).tolist() #to avoid different type returned
                if debug:
                    print(flux, ",")
            return flux-penalty(thresholding(x))*coefpenalty
        def myconstraint(x, grad):
            if grad.size > 0:
                grad[:]=grad_thresholding(x, eta=eta, beta=beta)
            return np.sum(thresholding(x))-constraintpores
        opt = nlopt.opt(nlopt.LD_MMA, 25)
        opt.set_lower_bounds(np.zeros(25))
        opt.set_upper_bounds(np.ones(25))
        opt.set_max_objective(myfunc)
        opt.set_xtol_rel(1e-3)
        opt.set_ftol_rel(1e-3)
        opt.add_inequality_constraint(myconstraint, 1e-8)
        x = opt.optimize(x1)
        maxf = opt.last_optimum_value()
        if np.all(np.isclose(thresholding(x1, eta=eta, beta=beta), thresholding(x, eta=eta, beta=beta))) or beta>2^16:
            return x, beta
        else:
            x1 = x
        beta = 2*beta
    return x1, beta
        

In [6]:
eta=0.2
beta = 1

In [8]:
xbest11= np.random.rand(25)
fbest11 = 0
x0s = []
for i in range(100):
    x0 = np.random.rand(25)
    x1, beta = optimization_withthresholdingNpenalty(x0, coefpenalty=1/25, constraintpores=10.5)
    optimum = thresholding(x1, eta=eta, beta=beta)
    flux_center.two_points_flux_normalized_grad(optimum)
    binarized = 1.*(thresholding(x1, eta=eta, beta=beta)>eta)
    f, _ = flux_center.two_points_flux_normalized_grad(binarized)
    if f>fbest11 and sum(binarized)<12:
        fbest11=f
        if fbest11>2.2:
            x0s.append(x0)
        xbest11 = x1

In [9]:
binarized = 1.*(thresholding(xbest11, eta=eta, beta=beta)>eta)
binarized

array([0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 0.])

In [10]:
xbest9= np.random.rand(25)
fbest9 = 0
x0s = []
for i in range(100):
    x0 = np.random.rand(25)
    x1, beta = optimization_withthresholdingNpenalty(x0, coefpenalty=1/25, constraintpores=8.5)
    optimum = thresholding(x1, eta=eta, beta=beta)
    flux_center.two_points_flux_normalized_grad(optimum)
    binarized = 1.*(thresholding(x1, eta=eta, beta=beta)>eta)
    f, _ = flux_center.two_points_flux_normalized_grad(binarized)
    if f>fbest9 and sum(binarized)<10:
        fbest9=f
        if fbest9>2.2:
            x0s.append(x0)
        xbest9 = x1

In [11]:
binarized = 1.*(thresholding(xbest9, eta=eta, beta=beta)>eta)
binarized

array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 1., 0., 1., 0.])

In [28]:
xbest7= np.random.rand(25)
fbest7 = 0
x0s = []
for i in range(100):
    x0 = np.random.rand(25)
    x1, beta = optimization_withthresholdingNpenalty(x0, coefpenalty=1/25, constraintpores=6.5)
    optimum = thresholding(x1, eta=eta, beta=beta)
    flux_center.two_points_flux_normalized_grad(optimum)
    binarized = 1.*(thresholding(x1, eta=eta, beta=beta)>eta)
    f, _ = flux_center.two_points_flux_normalized_grad(binarized)
    if f>fbest7 and sum(binarized)<7.5:
        print(sum(binarized), fbest7, binarized)
        fbest7=f
        if fbest7>2.2:
            x0s.append(x0)
        xbest7 = x1

7.0 0 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1.
 0.]


In [36]:
binarized = 1.*(thresholding(xbest7, eta=eta, beta=beta)>eta)
binarized

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 0.])